# WORD REPRESENTATION

* NLP:
* word embedding (geometry+arithmetic for words)
* debias word embeddings

## Word Representation:

One-hot notation: Having a vocabulary $V$, and "Man" $ = O_{5391}$, disadvantage: treats each word as a thing onto itself, and doesnt help the model to generalize across words: distances in one-hot aren't related to semantic (inner prod. between any two one-hot is zero, so distance is always same).

Instead, *featurized representation*: word embedding.


| FEAT | man | woman | king | queen | apple | orange |
| --- | --- | --- | --- | --- | --- | --- |
| Gender | -1 | 1 | -0.95 | 0.97 | 0.01 | 0 |
| Royal | 0.01 | 0.02 | 0.93 | 0.95 | 0 | -0.01 |
| Age | 0.03 | 0.02 | 0.7 | 0.69 | 0.03 | -0.01 |
| Food | 0.05 | 0 | 0.04 | 0.02 | 0.95 | 0.97 |

So now the vector for "Man", $e_{5391}$, will have as many dimensions as features has our embedding (usually 50 to 1000), and the model will have a better time finding semantic similarities.

* We will see how to learn embeddings: as in NNs, embedding features won't be that explicit.

* Visualizing embeddings: t-SNE

# USING WORD EMBEDDINGS

## NLP applications:

* "named entity recognition example": instead of one-hot, we pipe now the input words through an embedding before passing it to the BRNN (or model). Advantages:
  * NN will have an easier time learning generalizations, since changing "orange farmer" to "apple farmer" won't be a big gap
  * NN will react better to seldom words: even if the training set didn't contain them, the embedding may put them close to words that the RNN was trained on, so it knows what to do (transfer learning).
  * Embeddings are unsupervised and fast, can be trained on 1-100 billion words (quite reasonable).
  * The input vector becomes more dense (instead of 10k one-hot dimensions maybe becomes 300 dense features). Not always an advantage
    
Specific steps:

1. Learn word embedding from large text corpus, or use pre-trained one
2. Transfer embedding to new task with smaller training set
3. Optionally: continue finetuning the embeddings with new data (only if our dataset is big enough)

And 
* Embeddings have proven useful for: named entity recognition, text summarization, coreference resolution, parsing...
* They weren't as useful for: language modelling, machine translation, (especially if you already have lots of data for that task).

**Transfer learning**: by replacing the one-hots with the embeddings, algos generalize better and/or learn from less data. The key is the relation between the embedding's and the problem's data size: big-data based embeddings are useful usually if the training set is really small. If the training set is very big probably they don't bring much to the table. Also, if the training set is rather big, the embeddings can be finetuned (if small, don't bother).

## Finally

Relation to face encoding: for face recognition a CNN learns an "embedding" for a face and decides on the top of that if it is the same person. This is similar. **One difference: for face recognition you want the NN to be able to generalize to new, unseen pictures. Embeddings have a fixed vocabulary, and we learn one embedding per vocabulary element and that's it**.




# PROPERTIES OF WORD EMBEDDINGS

* Analogy reasoning: maybe not the most important but representative of embeddings: man $\rightarrow$ woman, king $\rightarrow$ ??

* Arithmetic: $e_{man}-e_{woman} \approx e_{king}-e_{queen}$. The difference in both is basically gender. So $e_{king}-e_{man}+e_{woman} \approx e_{queen}$. *Mikolov et a. 2013*. In general, find the word that maximizes similarity to (king-man+woman).
* Note that this arithmetic doesn't usually work on t-SNE because of its non-linear mapping.

## Similarity functions:

### Cosine similarity
$sim(u,v) = \frac{u^Tv}{\lVert u \rVert_2 \lVert v \rVert_2}$

will be 1 if both point to the same direction, drop to 0 if perpendicular and to -1 if opposed.

### Euclidean distance

$\lVert u-v \rVert^2$ a measurement of dissimilarity (has to be minimized). Works as well, less often used.


# EMBEDDING MATRIX

When learning a "word embedding", an "embedding matrix" is created.

The matrix $E$ has shape `(num_features, len(vocabulary))`. Note that $E \cdot o_{idx} = E_{:, idx} =: e_{idx} := e_w$

Initialize it randomly and optimize with gradient descent.

# LEARNING WORD EMBEDDINGS

Started with relatively complex, and got simpler with time realizing that still works. Follow this path to get a better intuition


## Embedding throug NN:

| I | want | a | glass | of | orange |
| --- | --- | --- | --- | --- | --- |
| 4343 | 9665 | 1 | 3852 | 6163 | 6257 |

**Building a language NN model is a reasonable way to train an embedding**:

* Bengio et al 2003: *A neural probabilistic language model*:
  1. Pass the one-hot words through the $E$ matrix to get the corresponding $e$ dense and less dimensioned vectors.
  2. Train a network to predict a word at a given step by feeding it with the concatenated e-vectors of the previous steps.
  3. The softmax output of the network is trained with BCE on the one-hot of the target. 
  
This way gradient descent not only forces the network to learn to predict the next word, but also **the E matrix has to embed the words efficiently, which implicitly takes semantics and grammar into account in a simple, linear way**.

## Towards simpler models:

It turns out that the "history of words" can be changed by other contexts: the N *surrounding words* also works, but **even just the previous word** works well (for embeddings, not for language models). This is the basis for the *skip gram* algorithm.

# WORD2VEC

Much simpler than learning a language model on the top of the embedding. Mikolov et al. 2013

## Skip-grams:

* **context-to-target mapping** to create a supervised learning problem:

Instead of picking a fixed one, pick a random origin and a random target, which can be before or after but not the same. The supervised learning problem on the top will be very unlikely solved, but it will learn a good word embedding.

## Skip-gram Model:

* Vocab size = 10k words (some have 1mi+)
* context "orange" $o_{orange} \rightarrow o_{juice}$ target mapping.
* $o_{orange} \rightarrow E \rightarrow e_{orange} \rightarrow Softmax \rightarrow o_{juice}$. Note that the softmax layer has its own weights and biases to be trained.
* Loss function is the BCE: $\mathcal{L(\hat y, y)} = - \sum y_i log(\hat y_i)$

* **problem**: computational speed at the softmax: to normalize, it has to  sum one exponent for every output unit (as many as vocab size). **solution**: hierarchical softmax. Instead of taking a whole softmax of size, say $2^a$, we can have a (kind of Huffman) tree of binary classificators with depth $a$.

Further details in the paper. No more time spent because there are simpler models (like *negative sampling*).

## How to sample the context?

There are words in the text like "the, of, a..." that happen very frequently, so there will be many mappings repeated which won't be very informative for the embedding. Ignoring them will make the training efficient and effective. So in practice, more common words have less probability to be sampled.


## CBOW (continuous bag of words):

In the paper referred before (Mikolov et al. 2013) this model is also discussed. It takes the surrounding words, has advantages and disadvantages. The main problem with skip-gram is the softmax bottleneck, which can be fixed.

# NEGATIVE SAMPLING

Modify skip-gram to overcome the time complexity problem and make it more efficient.

## Defining a new learning problem:

* Instead of training to predict word pairs, **we learn to predict whether a sampled pair is a "context target"** (orange->juice is, orange->king is not). For that
  1. pick two contiguous words in the text (orange->juice) and label them with 1
  2. pick $k$ many random targets (orange->?) and label them with 0. For large datasets, $k \in (2..5)$, smaller $(5-20)$.
  3. Input to the supervised model is the context-target matrix in $(k+1, 2)$, and output is a binary label: for the given context (orange), is this the valid target?
  4. Model is **one logistic regression per context word**: $P(y=1|c,t) = \sigma(\Theta_t^T e_c)$, where the logistic regression has k+1 dimensions.
  
  
**Instead of training a single softmax with 10000 dimensions, we train 10000 binary classifiers: but in each training step we train only k+1 of them, which is way more faster and the embedding is still being learned**.

## Selecting negative examples

We kave a k-to-1 negative ratio of examples. How to pick the k ones?, i.e. $P(w_i)$ (the probability of picking word $w_i$):
 * Uniform distribution would be $P = 1/|vocab|$
 * Observed distribution in corpus would be $f(w_i)$ (the word's frequency)
 
In the paper, they report a formula somewhere inbetween both, that shows good results and is widely used (theoretical justification maybe questionable):

$P(w_i) = \frac{f(w_i)^{\frac{3}{4}}}{\sum_{j=1}^{10000} f(w_j)^{\frac{3}{4}}}$

## Using an embedding:

There are open source implementations, and pre-trained embeddings


# GloVe word vectors:

* Global Vectors for word represntation (Pennington et al 2014)
* Even simpler
* Not so used as skip-gram and word2vec, but has some momentum.

## How it works:

* Given a definition of what a context word and its target may be, count in a corpus how many times does word $t$ appear in the context of $c$, and put that in the matrix $X$.

* So $X_{ij}$ captures how often in our corpus does $j$ happen in the context of $i$. Depending on our definition of context, the matrix $X$ may be symmetrical.

* The model is then to minimize the following quadratic formula:

$\sum_{i=1}^{10000} \sum_{j=1}^{10000} f(X_{ij})(\Theta_i^T e_j + b_i + b_j - log(X_{ij}))^2$

Note the following:
* the $f$ weighting function is defined to be 0 if $X_{ij}=0$. So if a combination doesn't occur in the corpus, it is simply ignored.

* log term: for 1 occurrence, the term is zero. For 3, it subtracs something above 1, for 9 something above 2... etc. This means that the affine transformation from before ideally has to return $log(X_{ij}$ to have a minimal cost. The affine transformation then has to have a linearly bigger response for exponentially bigger occurrences.

* Note that for the non-zero occurrences, they will be words much more frequent than others (the, a, this...) and this could skew our minimization objective. Specific weighting heuristics can be applied (more details in the paper), to adapt the impact for the affine transformation error to the actual relevance of the regarded pair.

* The idea is that $E$ provides the embeddings in a way that makes easier for $\Theta$ to predict how often will a pair occur, and for that the embedding has to be efficient. But in fact, $\Theta$ and $E$ are interexchangeable in this formula, and once the minimization is performed, **their average $\frac{e_w + \Theta_w}{2}$ can be taken as final result**.

## A note on the featurization view of word embeddings:

* It can't be garanteed that the features of embedded vectors have an interpretable meaning (unlike our king/queen table discussed at beginning), and finding interpretations in that space may be very difficult.

# Sentiment Classification:

* Take a piece of text by a person, and identify whether they like or dislike the thing that they are talking about (f.e. input is a sequence of words, output may be star rating from 1 to 5).

* Relevant NLP task. Challenge: usually small training sets (10k to 100k words not uncommon). Embeddings can help

## A simple model:

1. Pick the sequence as one-hot
2. convert it to a sequence of embedded vectors (using a pre-trained embedding)
3. Average the embedded vectors (helps making the model independent of the sequence's length)
4. Feed the averaged embedding to a softmax classifier to predict the star rating distribution

Problem: averaging is not a good approach since some phrases aren't linear: lacking good service, good food, good etc... lots of "good".

# Using RNNs:

Feed the sequence as already seen to the RNN, and after the last step perform the softmax prediction (many-to-one setup).

This will be pretty decent, and with the embedding, training for "lacking" and then evaluating on "absent" still will work if they are embedded rather close.

# Debiasing Word Embeddings


Importance of removing language bias from embeddings to prevent reflecting gender, ethnicity, age, sexual orientation... examples (Bolubkasi et al 2016):

* Man->computer_programmer, woman->homemaker
* Father->doctor, mother->nurse

The embedding captures the text's biases. In both cases we are far from perfect, but removing the bias from all writers seems harder than fixing it for the machine learning algorithms. A simplified setup (applied to gender bias in this example) would be:

1. Identify bias direction (f.e. by averaging $[e_{she}-e_{he}, e_{mother}-e_{father}...]$ all gender-involved terms). The orthogonal directions (bias_dims - 1) are non-bias directions. Instead of averaging actually SVD is used.

2. Neutralize: for every word that is not **definitional**, project onto the non-bias hyperplane to get rid of the component in the bias direction.

3. Equalize pairs: all the words that are gender-involved should keep equal distance to those neutralized words, so shift (grandmother, grandfather) across the bias component until they are equidistant to babysitter, doctor etc.

4. Which words to neutralize and equalize? (f.e. we don't want to neutralize "catholic_priest" if they are always men. Also, we don't want to equalize for "beard" since it should be closer to men). In the paper they use a classifier on english language to find definitional words, and it turns out most words are non-definitional. So handpicking the definitional seems an OK approach, but this is still open research




# ASSIGNMENTS: